In [16]:
#install latest Chrome driver to your machine https://sites.google.com/chromium.org/driver/
#install Beautiful Soup 
#install selenium

#imports
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json 
from selenium.webdriver.chrome.service import Service

#base product URL
URL = "https://e-katanalotis.gov.gr/product/"
min_id = 1002
max_id = 1003

#creates a list [min_id...max_id-1]
productids = range(min_id, max_id)

#final JSON
results = {}
results['fetch_date']=int(time.time())
results['data']=[]

#get product information
for productid in productids:
    print(productid)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    # executable_path param is where the Chrome driver is installed
    s=Service('Users\kamin\Downloads\chromedriver_win32\chromedriver')
    browser = webdriver.Chrome(options=options, service = s)
    browser.get(URL+str(productid))

    #get dates from price chart
    date_data = browser.execute_script('return Highcharts.charts[0].series[0].data.map(x => x.category)')
    #append year to date labels
    date_data = [s +'/2022' for s in date_data]

    #get prices from price chart
    price_data = browser.execute_script('return Highcharts.charts[0].series[0].data.map(x => x.y)')

    #find product name
    html = browser.page_source
    soup = BeautifulSoup(html, features="html5lib")

    #test html rendering
    #print(soup.prettify())

    pname = soup.find('p', attrs={'class':'product-name'}).text 
    browser.quit()

    #transform dates

    for i in range(len(date_data)-1):
        dparts = date_data[i].split("/")
        newd = '2022-'+dparts[1]+'-'+dparts[0]
        date_data[i]=newd
        
    #create a result object
    result = {}
    result['id']=productid
    result['name']=pname
    result['prices']=[]
    for i in range(len(date_data)-1):
        result['prices'].append({'date':date_data[i], 'price':price_data[i]})
    
    #append it to the list
    results['data'].append(result)

print('Done')

1002


JavascriptException: Message: javascript error: Cannot read properties of undefined (reading 'series')
  (Session info: headless chrome=111.0.5563.146)
Stacktrace:
Backtrace:
	(No symbol) [0x009BDCE3]
	(No symbol) [0x009539D1]
	(No symbol) [0x00864DA8]
	(No symbol) [0x008676FB]
	(No symbol) [0x008675D0]
	(No symbol) [0x00867EB0]
	(No symbol) [0x008BDEED]
	(No symbol) [0x008AAECC]
	(No symbol) [0x008BD57C]
	(No symbol) [0x008AACC6]
	(No symbol) [0x00886F68]
	(No symbol) [0x008880CD]
	GetHandleVerifier [0x00C33832+2506274]
	GetHandleVerifier [0x00C69794+2727300]
	GetHandleVerifier [0x00C6E36C+2746716]
	GetHandleVerifier [0x00A66690+617600]
	(No symbol) [0x0095C712]
	(No symbol) [0x00961FF8]
	(No symbol) [0x009620DB]
	(No symbol) [0x0096C63B]
	BaseThreadInitThunk [0x762500F9+25]
	RtlGetAppContainerNamedObjectPath [0x77A87BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77A87B8E+238]


In [ ]:
#inspect final JSON
json_object = json.dumps(results, indent = 2, ensure_ascii=False).encode('utf8')
print(json_object.decode())

{
  "fetch_date": 1680104205,
  "data": []
}


In [ ]:
#save final JSON to file
with open("output_"+str(productids[0])+"_"+str(productids[len(productids)-1])+"_"+str(results['fetch_date'])+".json", "w", encoding='utf-8') as outfile:
    json.dump(results, outfile, ensure_ascii=False ,indent=2)